In [1]:
import numpy as np
import numpy.random as random
import scipy as sp
#import modin.pandas as pd #多核處裡
import pandas as pd 
from pandas import Series,DataFrame

import datetime

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import ast
import csv

import sklearn
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.utils import resample
from sklearn import metrics
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

%matplotlib inline

%precision 3

'%.3f'

In [2]:
def stringConvert(string, punc):
    index = string.find(punc)
    num = string[1:index]
    return num
def removePercent(s):
    s = float(s[:-1])
    return s
def getPrecision(x,y,propotion,y_num,estimator):
    proba = estimator.predict_proba(x)[:,0]#得到預測機率
    rankIndex = np.flip(np.argsort(proba))
    num = int(len(rankIndex)*propotion)    
    rankIndex = rankIndex[:num]#取得風險機率最高前5%的資料其在原本df的index
    
    
    fraudCount = 0
    for key in rankIndex:
        if(y[key] == 1):
            fraudCount+=1#如果它真的是詐欺
            
    result = fraudCount/y_num
    
    return result
        
    


In [ ]:
# merge files
df = pd.DataFrame()
for i in range(1,7):
    globals()["df"+str(i)] = pd.read_csv("./data/AG_PROFILE_201912_FIN_"+str(i)+".csv",thousands=',',encoding='big5',engine='python')
    df = df.append(globals()["df"+str(i)])
df.to_csv('output.csv')

In [ ]:
df = pd.read_csv('output.csv',index_col=0, thousands=',')
df.shape #應為391行

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
num_features=[]
object_features=[]
for dtype,feature in zip(df.dtypes,df.columns):
        if dtype =='int64' or dtype=='float64':
            num_features.append(feature)
        #elif dtype=='object':
        else:
            object_features.append(feature)
            
print(f'{len(num_features)} Numeric Features: ') #{num_features}\n')
print(f'{len(object_features)} Object Featurs: ')#{object_features}\n')        

In [ ]:
pd.set_option('display.max_columns', None)
df[object_features].head()

In [ ]:
#labelencoder
df["area_name"] = [stringConvert(i, "_") for i in df.area_name]
df["AG_GP"] =  [stringConvert(i, ".") for i in df.AG_GP]
df["CLASS"] = [stringConvert(i, "_") for i in df.CLASS]

df['office_CITY'] = LabelEncoder().fit_transform(df['office_CITY'])

df['area_name']

In [ ]:
#object feature裡有些是百分比有些是數值
percent_features = object_features[4:]
#df_percent = pd.DataFrame()

for col in percent_features:
    percent_num = df[col].str.contains('%').value_counts().values[0]
    percent_check = df[col].str.contains('%').value_counts().index[0]
    print("percent_num is" , percent_num)
    print("percent_check is", percent_check)
    if(percent_check == True and percent_num > 10):
        print(col+' contains percent')
        #df_percent[col] = [removePercent(s[1:-1].strip())*(-0.01) if s[0]=='(' else removePercent(s)*0.01 for s in df[col]]
        df[col] = [removePercent(s[1:-1].strip())*(-0.01) if s[0]=='(' else removePercent(s)*0.01 for s in df[col]]
    else:
        print(col+ ' contains float')

In [ ]:
for c in object_features :
    print(c)
    for k,v in zip(df[c].index,df[c]):
        if(type(v) == str):
            df[c][k] = float(v.replace(',',''))

In [ ]:
#處理剩下的object type的"數字"資料
import sys
 
sys.setrecursionlimit(1000000) 
for col in object_features:
    print(col)
    df[col] = [float(s.replace(',','')) if type(s) == str else s for s in df[col] ]         

In [ ]:
df.to_csv('clean_data.csv')

In [42]:
df = pd.read_csv('clean_data.csv',index_col=0)

In [ ]:
df['Y'].value_counts()

In [43]:
#遺漏值尋找
missing = []
num = df.isna().sum()#pd.series type
for k,v  in zip(num.index,num.values):
     if(v > 0):
          print(k,v)  
          missing.append(k)

CLIENT_INCOME 1428
CHN_AG_RECENCY 13108
CALLOUT_AG_RECENCY 24953
Y_RECENCY_HIS 18759
Y1_RECENCY_HIS 28713
Y2_RECENCY_HIS 21486
Y3_RECENCY_HIS 24563


In [44]:

df['CLIENT_INCOME'] = df['CLIENT_INCOME'].fillna(df['CLIENT_INCOME'].median())
df['CLIENT_INCOME'].isnull().value_counts()

False    28572
True      1428
Name: CLIENT_INCOME, dtype: int64

In [45]:
for i in missing:
    df[i] = df[i].fillna(-1)

In [46]:
#確認是否還有遺漏值
num = df.isna().sum()#pd.series type
for k,v  in zip(num.index,num.values):
     if(v > 0):
          print(k,v)  


In [ ]:
y_train=df['Y']
x_train=df.drop(['Y'],axis = 1)

sm = SMOTE(random_state=11)
x_res, y_res = sm.fit_sample(x_train,y_train)
print(y_res.value_counts())

In [ ]:
x_res,y_res = BorderlineSMOTE(random_state=42, kind='borderline-2').fit_resample(x_train, y_train)

In [ ]:
important_features = list(feats[:28].index)


start = datetime.datetime.now()
gbdt.fit(x_res[important_features],y_res)
end = datetime.datetime.now()

print("running time : ",end-start)



In [ ]:

#print("auc",metrics.roc_auc_score(x_res[important_features],y_res))



In [ ]:

print(y_res.value_counts())

In [ ]:
count_class_0, count_class_1 = df['Y'].value_counts()


df_class_0 = df[df['Y'] == 0]#class 0
df_class_1 = df[df['Y'] == 1]#class 1
df_class_0_under = df_class_0.sample(count_class_1*10)#randomly sample 2000 data from class_O 
df_bal = pd.concat([df_class_0_under, df_class_1], axis=0)#merge


df_bal['Y'].value_counts().plot(kind='bar', title='Count')


In [ ]:
y_res = df_bal['Y']
x_res = df_bal.drop(['Y'],axis = 1)

In [3]:
#該資料已進行遺漏值填補但未進行oversampling
df = pd.read_csv('train.csv',engine='python')
df.shape

(30000, 265)

In [4]:
pd.set_option('display.max_columns', None)
df.head()

,AGENT_RK,AG_AGE,AG_TENURE,SEX,CLIENT_INCOME,total_aum,AG_LOAN_POL_CNT_HIS,RELATION_POL_CNT,RELATION_CUST_CNT,CHN_AG_CNT_HIS,CHN_AG_CNT_12M,CHN_AG_RATE_12M,CALLOUT_AG_CNT_HIS,CALLOUT_POL_CNT_HIS,CALLOUT_POL_RATE_HIS,CALLOUT_AG_CNT_12M,CALLOUT_POL_CNT_12M,CALLOUT_AG_RATE_12M,CALLOUT_POL_RATE_12M,Y_CNT_HIS,Y1_CNT_HIS,Y2_CNT_HIS,Y3_CNT_HIS,Y_RISK_FLG,Y1_RISK_FLG,Y2_RISK_FLG,Y3_RISK_FLG,HOLD_CUST_CNT,HOLD_CUST_CNT_HIS,AVG_AFYP_HOLD,AVG_POL_CNT_HOLD,AVG_POL_CNT_HOLD_HIS,INS_CUST_CNT,INS_CUST_CNT_HIS,AVG_AFYP_INS,AVG_POL_CNT_INS,AVG_POL_CNT_INS_HIS,G0_CNT_AG,G1_CNT_AG,G2_CNT_AG,G3_CNT_AG,G4_CNT_AG,TOT_CNT_AG,FAN_CUST_CNT,FAN_CUST_RATE,EVER_VIP_CNT,EVER_VIP_CNT_HIS,EVER_VIP_RATE,EVER_VIP_RATE_HIS,UNDERTAKE_CUST_CNT,UNDERTAKE_CUST_CNT_HIS,UNDERTAKE_CUST_RATE,UNDERTAKE_CUST_RATE_HIS,PURCHASE_CUST_CNT_1YR,PURCHASE_CUST_CNT_2YR,PURCHASE_CUST_CNT_3YR,PURCHASE_CUST_RATE_1YR,PURCHASE_CUST_RATE_2YR,PURCHASE_CUST_RATE_3YR,POL_1_CNT,POL_3_CNT,POL_4_CNT,POL_1_RATE,POL_3_RATE,POL_4_RATE,CONTACT_1_CNT,CONTACT_1_RATE,FOREIGN_CUR_CUST_CNT,FOREIGN_CUR_CUST_RATE,FAIL_POL_CUST_CNT,FAIL_POL_CUST_RATE,ADDRESS_3_POL_RATE,ADDRESS_4_POL_RATE,ADDRESS_5_POL_RATE,ADDRESS_2_CUST_RATE,ADDRESS_3_CUST_RATE,ADDRESS_4_CUST_RATE,PHONE_3_POL_RATE,PHONE_4_POL_RATE,PHONE_5_POL_RATE,PHONE_2_CUST_RATE,PHONE_3_CUST_RATE,PHONE_4_CUST_RATE,CELL_3_POL_RATE,CELL_4_POL_RATE,CELL_5_POL_RATE,CELL_2_CUST_RATE,CELL_3_CUST_RATE,CELL_4_CUST_RATE,AG_CELL_SAME_POL_CNT,AG_CELL_SAME_CUST_CNT,AGRELATION_ADDRESS_SAME_POL_CNT,AGRELATION_ADDRESS_SAME_CUST_CNT,AGRELATION_PHONE_SAME_POL_CNT,AGRELATION_PHONE_SAME_CUST_CNT,AGRELATION_CELL_SAME_POL_CNT,AGRELATION_CELL_SAME_CUST_CNT,selfpay_cnt,selfpay_RATE,selfpay_REG_cnt,selfpay_REG_RATE,NEW_CUST,NEW_CUST_RATE,LOAN_POL_CNT_HIS,LOAN_POL_RATE,LOAN_CUST_CNT_HIS,LOAN_CUST_RATE,APL_POL_CNT_HIS,APL_POL_RATE,APL_CUST_CNT_HIS,APL_CUST_RATE,VALID_CNT_12M,INVALID_CNT_12M,INVALID_CNT_RATE_12M,DISCON_CNT_12M,DISCON_CNT_RATE_12M,TERMINATION_CNT_12M,TERMINATION_CNT_RATE_12M,ST_INVALID_CNT_12M,ST_INVALID_CNT_RATE_12M,ST_DISCON_CNT_12M,ST_DISCON_CNT_RATE_12M,ST_TERMINATION_CNT_12M,ST_TERMINATION_CNT_RATE_12M,CNACEL_CNT_HIS,ST_CNACEL_CNT_12M,CNACEL_CNT_RATE,ST_CNACEL_CNT_RATE_12M,FAIL_CNT_HIS,FAIL_CNT_RATE,DISCON_CNT_HIS,DISCON_CNT_RATE,TERMINATION_CNT_HIS,TERMINATION_CNT_RATE,TAKE_CNT_HIS,TAKE_CNT_RATE,TAKE_CNT_12M,TAKE_CNT_RATE_12M,UNDERTAKE_CNT_HIS,UNDERTAKE_CNT_RATE,UNDERTAKE_CNT_12M,UNDERTAKE_CNT_RATE_12M,FOREIGN_CNT_HIS,FOREIGN_CNT_RATE,FOREIGN_CNT_12M,FOREIGN_CNT_RATE_12M,TOT_CNT_12M_HIS,CNT_12M_RATE_HIS,TOT_AFYP_12M_HIS,AFYP_12M_RATE_HIS,AVG_AFYP_12M_HIS,POLCNT_TREND_FLG,AFYP_TREND_FLG,POL_RATE_0M,POL_RATE_1M,POL_RATE_2M,POL_RATE_3M,POL_RATE_4M,POL_RATE_5M,POL_RATE_MAX,POL_RATE_JUMP_FLG,POL_RATE_JUMP_FLG2,REG_CNT,SIN_CNT,ILP_CNT,AH_CNT,REG_CNT_RATE,SIN_CNT_RATE,ILP_CNT_RATE,AH_CNT_RATE,REG_AFYP,SIN_AFYP,ILP_AFYP,AH_AFYP,REG_AFYP_RATE,SIN_AFYP_RATE,ILP_AFYP_RATE,AH_AFYP_RATE,AVG_AFYP_REG,AVG_AFYP_SIN,AVG_AFYP_ILP,AVG_AFYP_AH,TOT_CNT,TOT_AFYP,AVG_AFYP,TOT_CNT_HIS,TOT_AFYP_HIS,AVG_AFYP_HIS,AHa_CNT_HIS,REG_CNT_12M_HIS,SIN_CNT_12M_HIS,ILP_CNT_12M_HIS,AH_CNT_12M_HIS,REG_CNT_RATE_12M_HIS,SIN_CNT_RATE_12M_HIS,ILP_CNT_RATE_12M_HIS,AH_CNT_RATE_12M_HIS,REG_AFYP_12M_HIS,SIN_AFYP_12M_HIS,ILP_AFYP_12M_HIS,AH_AFYP_12M_HIS,REG_AFYP_RATE_12M_HIS,SIN_AFYP_RATE_12M_HIS,ILP_AFYP_RATE_12M_HIS,AH_AFYP_RATE_12M_HIS,AVG_AFYP_REG_12M_HIS,AVG_AFYP_SIN_12M_HIS,AVG_AFYP_ILP_12M_HIS,AVG_AFYP_AH_12M_HIS,CLAIM_NO_CNT,CLAIM_POL_CNT,CLAIM_POL_RATE,CLAIM_CUST_CNT,CLAIM_CUST_RATE,AVG_CLAIM_NO_AMT,AVG_CLAIM_POL_AMT,AVG_CLAIM_CUST_AMT,CLAIM_NO_REJECT_CNT,CLAIM_NO_REJECT_RATE,AHa_CLAIM_POL_CNT,AHa_CLAIM_POL_CNT2,AHa_CLAIM_POL_RATE,AHa_CLAIM_POL_RATE2,CLAIM_NO_CNT_12M,CLAIM_POL_CNT_12M,CLAIM_POL_RATE_12M,CLAIM_CUST_CNT_12M,CLAIM_CUST_RATE_12M,CLAIM_NO_TREND_FLG,CLAIM_POL_TREND_FLG,CLAIM_CUST_TREND_FLG,AVG_CLAIM_NO_AMT_12M,AVG_CLAIM_POL_AMT_12M,AVG_CLAIM_CUST_AMT_12M,CLAIM_NO_REJECT_CNT_12M,CLAIM_NO_REJECT_RATE_12M,AHa_CLAIM_POL_CNT_12M,AHa_CLAIM_POL_CNT2_12M,AHa_CLAIM_POL_RATE_12M,AHa_CLAIM_

In [5]:
df['Y'].value_counts()

0    29805
1      195
Name: Y, dtype: int64

In [47]:
df_majority = df[df.Y==0]
df_minority = df[df.Y==1]

df_minority_upsampled = resample(df_minority, replace=True, n_samples=29805, random_state=123)
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

In [48]:
df_upsampled = df_upsampled.reset_index(drop = True)

In [49]:
y_res = df_upsampled['Y']
x_res = df_upsampled.drop(['Y'],axis = 1)

In [ ]:
x_res

In [50]:
print(y_res.value_counts())
print(len(x_res))

0    29805
1    29805
Name: Y, dtype: int64
59610


In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=15, criterion="entropy", max_depth=10)
dt.fit(x_res,y_res)
#gbdt = GradientBoostingClassifier()
#gbdt.fit(x_res,y_res)

#lr  = LogisticRegression()
#lr.fit(x_res,y_res)

#rfc = RandomForestClassifier()
#rfc.fit(x_res,y_res)


start = datetime.datetime.now()
feats = pd.Series(data=dt.feature_importances_, index=x_res.columns)
end = datetime.datetime.now()
feats = feats.sort_values(ascending=False)
print("running time : ",end-start)
feats[:28]

running time :  0:00:00.000991


Y1_RECENCY_HIS            0.191962
INVALID_CNT_9M            0.117547
selfpay_RATE              0.066661
SIN_AFYP_RATE             0.032861
AHa_CNT_HIS               0.030601
selfpay_REG_RATE          0.028453
INVALID_CNT_RATE_3M       0.026967
TOT_AFYP_6M_HIS           0.025496
CLAIM_NO_CNT_9M           0.024323
REG_AFYP_RATE             0.023965
CLAIM_NO_REJECT_RATE      0.022825
ADDRESS_3_POL_RATE        0.022512
AVG_AFYP_AH               0.022130
INVALID_CNT_RATE_6M       0.021916
ADDRESS_2_CUST_RATE       0.021632
POL_3_RATE                0.021616
ADDRESS_4_CUST_RATE       0.021271
PHONE_2_CUST_RATE         0.019448
AH_AFYP_RATE_12M_HIS      0.018838
TERMINATION_CNT_HIS       0.018830
CLAIM_POL_CNT_6M          0.017411
PURCHASE_CUST_CNT_3YR     0.016358
REG_CNT_RATE              0.015897
AVG_AFYP_REG              0.014712
TOT_CNT_HIS               0.014311
total_aum                 0.014195
POL_4_RATE                0.013072
AHa_CLAIM_POL_RATE2_6M    0.011873
dtype: float64

In [57]:
feats = feats.sort_values(ascending=False)
important_features = feats[:100].index
#gbdt = GradientBoostingClassifier()
#gbdt.fit(x_res[important_features],y_res)

#dt = DecisionTreeClassifier(random_state=15, criterion="entropy", max_depth=10)
#dt.fit(x_res[important_features],y_res)
lr.fit(x_res[important_features],y_res)
iteration = np.linspace(0,1,21)
for i in iteration:
    print("percent {:.2f}%".format(i*100))
    print("precision {:.4f}".format(getPrecision(x_res[important_features],y_res,i,29805,lr)))
    print('-----------------')

C:\Users\Hubert\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


percent 0.00%
precision 0.0000
-----------------
percent 5.00%
precision 0.0307
-----------------
percent 10.00%
precision 0.0663
-----------------
percent 15.00%
precision 0.1024
-----------------
percent 20.00%
precision 0.1401
-----------------
percent 25.00%
precision 0.1803
-----------------
percent 30.00%
precision 0.2305
-----------------
percent 35.00%
precision 0.2614
-----------------
percent 40.00%
precision 0.2951
-----------------
percent 45.00%
precision 0.3471
-----------------
percent 50.00%
precision 0.3939
-----------------
percent 55.00%
precision 0.4480
-----------------
percent 60.00%
precision 0.4957
-----------------
percent 65.00%
precision 0.5485
-----------------
percent 70.00%
precision 0.6116
-----------------
percent 75.00%
precision 0.6686
-----------------
percent 80.00%
precision 0.7319
-----------------
percent 85.00%
precision 0.7866
-----------------
percent 90.00%
precision 0.8611
-----------------
percent 95.00%
precision 0.9306
-----------------
pe

########

In [ ]:
y_res[11435]

In [22]:
feats[:28]

Y1_CNT_HIS                 0.084173
INVALID_CNT_RATE_12M       0.076151
Y_CNT_HIS                  0.073549
FAIL_CNT_HIS               0.049804
SIN_AFYP_RATE              0.033358
Y_RECENCY_HIS12            0.024665
selfpay_RATE               0.024653
DISCON_CNT_RATE            0.018725
Y2_CNT_HIS                 0.017772
TERMINATION_CNT_HIS        0.016700
total_aum                  0.015797
AVG_AFYP_REG               0.015457
ADDRESS_2_CUST_RATE        0.015038
TERMINATION_CNT_RATE       0.012959
ST_INVALID_CNT_RATE_12M    0.012566
AVG_AFYP_AH                0.011513
TAKE_CNT_RATE              0.011340
G4_CNT_AG                  0.011213
CELL_4_POL_RATE            0.010333
PHONE_2_CUST_RATE          0.009951
APL_POL_RATE               0.009810
REG_AFYP                   0.009625
CLIENT_INCOME              0.009300
HOLD_CUST_CNT_HIS          0.009071
FAN_CUST_CNT               0.008591
CLAIM_CUST_RATE_12M        0.007646
SIN_AFYP                   0.007377
AVG_AFYP_REG_12M_HIS       0